**Q1**

In [4]:
import os
import kagglehub

# 1. Set the environment variable to your desired path
custom_path = "G:\class codes\cse440_NLP\lab 2" # Replace with your actual path
os.environ["KAGGLEHUB_CACHE"] = custom_path

# 2. Download the dataset (it will now use the custom path)
# Note: The function will return the path to the downloaded resource within your custom cache.
path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to downloaded dataset:", path)

Path to downloaded dataset: G:\class codes\cse440_NLP\lab 2\datasets\lakshmi25npathi\imdb-dataset-of-50k-movie-reviews\versions\1


In [5]:
import pandas as pd
# Load the dataset
data = pd.read_csv("datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews/versions/1/IMDB Dataset.csv")
print(data.head())
# print(data['review'][0])

                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive


In [6]:
from nltk.tokenize import word_tokenize
q1_corpus = data['review'].tolist()
q1_labels = data['sentiment'].tolist()
# print(q1_corpus[0].lower())
#preproccess
punctutations=['.',',','!','?',';',':','"',"'",'(',')','[',']','{','}','-','_','/','\\','@','#','$','%','^','&','*','~','`','<','>','=','+','br']
for i in range(len(q1_corpus)):
    q1_corpus[i]=word_tokenize(q1_corpus[i].lower())
    for word in q1_corpus[i]:
        if word in punctutations:
            q1_corpus[i].remove(word)

def sentence_joiner(tokenized_sentence):
    sentence=""
    for word in tokenized_sentence:
        sentence+=word+" "
    return sentence.strip()
bow=[]
for i in range(len(q1_corpus)):
    bow.append(sentence_joiner(q1_corpus[i]))


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(bow)
vocab = vectorizer.get_feature_names_out()
print("Vocabulary Size:", len(vocab))
print("Dimension: ",X.shape)  

Vocabulary Size: 101881
Dimension:  (50000, 101881)


**Q2**

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000)
X_tfidf = tfidf_vectorizer.fit_transform(bow)
vocab_tfidf = tfidf_vectorizer.get_feature_names_out()
print("TF-IDF Vocabulary Size:", len(vocab_tfidf))
print("TF-IDF Matrix Shape:", X_tfidf.shape)

TF-IDF Vocabulary Size: 10000
TF-IDF Matrix Shape: (50000, 10000)


In [24]:
dframe=pd.DataFrame(X_tfidf.toarray(), columns=vocab_tfidf)
dframe.head()

,00,000,007,10,100,1000,101,11,12,13,...,zhang,zizek,zoey,zombi,zombie,zombies,zone,zoom,zorro,zucco
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.056852,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.208289,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [9]:

import numpy as np

max_scores = np.array(X_tfidf.max(axis=0).todense()).flatten()
top_max_indices = max_scores.argsort()[-10:][::-1]

print("Top 10 words by maximum TF-IDF score:")
for i, idx in enumerate(top_max_indices):
    print(f"{i+1}. {vocab_tfidf[idx]}: {max_scores[idx]:.4f}")




Top 10 words by maximum TF-IDF score:
1. robot: 0.9157
2. ernest: 0.9054
3. wine: 0.8780
4. woo: 0.8747
5. steve: 0.8704
6. bad: 0.8675
7. custer: 0.8668
8. demons: 0.8573
9. muppet: 0.8563
10. jokes: 0.8530


# **Q3**

In [10]:
def load_glove_model(glove_file):
    word_vectors = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors
glove_file = 'glove.6B.100d.txt'  # Ensure this file is in the working directory
glove_model = load_glove_model(glove_file)


In [11]:
teacher=glove_model['teacher']
educate=glove_model['educate']
heal=glove_model['heal']
doctor=glove_model['doctor']
custom=teacher - educate + heal


In [12]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
doctor_customer_similarity = cosine_similarity([custom], [doctor])
temp=round(float(doctor_customer_similarity[0][0]),4)
print(f"Cosine similarity between 'doctor' and the custom(teacher-educate+heal) vector: {temp}")

Cosine similarity between 'doctor' and the custom(teacher-educate+heal) vector: 0.5419


# **Q4**

In [13]:
import nltk
nltk.download('brown')
from nltk.corpus import brown
brown.fileids()
brown_text= brown.words('ca01')
print(brown_text[100:500])
lowered_brown_text = [word.lower() for word in brown_text]



['Allen', 'Jr.', '.', '``', 'Only', 'a', 'relative', ...]


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\nooba\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [14]:
!pip install gensim
import gensim
from gensim.models import Word2Vec

In [15]:
model_skipgram=Word2Vec(vector_size=100, window=5, sg=1, min_count=1)
model_cbow=Word2Vec(vector_size=100, window=5, sg=0, min_count=1)

In [16]:
vocabulary_skip=model_skipgram.build_vocab([lowered_brown_text])
model_skipgram.train([lowered_brown_text], total_examples=len(lowered_brown_text), epochs=10)
vocabulary_cbow=model_cbow.build_vocab([lowered_brown_text])
model_cbow.train([lowered_brown_text], total_examples=len(lowered_brown_text), epochs=10)

(15587, 22420)

In [17]:
teacher_vec_skipgram=model_skipgram.wv['teacher']
teacher_vec_cbow=model_cbow.wv['teacher']
school_vec_skipgram=model_skipgram.wv['school']
school_vec_cbow=model_cbow.wv['school']
jury_vec_skipgram=model_skipgram.wv['jury']
jury_vec_cbow=model_cbow.wv['jury']
court_vec_skipgram=model_skipgram.wv['court']
court_vec_cbow=model_cbow.wv['court']

custom2_skip=teacher_vec_skipgram - school_vec_skipgram + court_vec_skipgram
custom2_cbow=teacher_vec_cbow - school_vec_cbow + court_vec_cbow

In [18]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
jury_custom2_skip_similarity = cosine_similarity([custom2_skip], [jury_vec_skipgram])
jury_custom2_cbow_similarity = cosine_similarity([custom2_cbow], [jury_vec_cbow])
print(f"Cosine similarity between 'jury' and the custom2_skip(teacher-school+court) vector: {jury_custom2_skip_similarity[0][0]:.4f}")
print(f"Cosine similarity between 'jury' and the custom2_cbow(teacher-school+court) vector: {jury_custom2_cbow_similarity[0][0]:.4f}")

Cosine similarity between 'jury' and the custom2_skip(teacher-school+court) vector: 0.9816
Cosine similarity between 'jury' and the custom2_cbow(teacher-school+court) vector: 0.1293
